# CellLogger Demo

In [1]:
%reload_ext autoreload
%autoreload 2

# Setup

In [2]:
import numpy as np
import torch
from ipyexperiments import *

In [3]:
def consume_cpu_ram(n): return np.ones((n, n))
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_cpu_ram_128mb(): return consume_cpu_ram(2**12)
def consume_gpu_ram_256mb(): return consume_gpu_ram(2**13)

# Main functionality

In [4]:
exp1 = IPyExperimentsPytorch(exp_enable=False)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX TITAN X (12212 RAM)

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.000
･ CPU:          0          0      1,327 MB |
･ GPU:          0          0      4,702 MB |


In [5]:
# consume, consume more
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.024
･ CPU:        256          0      1,583 MB |
･ GPU:          0          0      4,702 MB |


In [6]:
# consume, consume more, then release some (temporarily requires double the memory)
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()
del a1

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.035
･ CPU:        128        128      1,455 MB |
･ GPU:          0          0      4,702 MB |


In [7]:
# consume, release, then consume more (half the memory is needed at the peak)
a1 = consume_cpu_ram_128mb()
del a1
a2 = consume_cpu_ram_128mb()

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.028
･ CPU:        128          0      1,455 MB |
･ GPU:          0          0      4,702 MB |


In [8]:
# consume, consume more, release all (all memory get reclaimed)
b1 = consume_cpu_ram_128mb()
b2 = consume_cpu_ram_128mb()
del b1
del b2

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.036
･ CPU:          0        256      1,455 MB |
･ GPU:          0          0      4,702 MB |


In [9]:
# same on gpu
c1 = consume_gpu_ram_256mb()
c2 = consume_gpu_ram_256mb()
del c1

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.157
･ CPU:          0          0      1,456 MB |
･ GPU:        256        256      4,958 MB |


In [10]:
# direct data accessor (in unrounded Bytes)
cpu_mem   = exp1.cl.data.cpu
gpu_mem   = exp1.cl.data.gpu
time_data = exp1.cl.data.time
cpu_mem
gpu_mem
time_data
gpu_mem.used_delta

CellLoggerMemory(used_delta=5517, peaked_delta=66648, used_total=1526886400)

CellLoggerMemory(used_delta=268435456, peaked_delta=268435456, used_total=5198905344)

CellLoggerTime(time_delta=0.15741968154907227)

268435456

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.010
･ CPU:          0          0      1,456 MB |
･ GPU:          0          0      4,958 MB |


In [11]:
exp1.cl.stop()

･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.006
･ CPU:          0          0      1,456 MB |
･ GPU:          0          0      4,958 MB |


In [12]:
# there should be no log appearing, after the logger has been stopped
a = consume_cpu_ram_128mb()

# Automatically reset NRG seed

In [13]:
exp2 = IPyExperimentsPytorch(exp_enable=False, cl_set_seed=42, cl_compact=True)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX TITAN X (12212 RAM)

CPU: 0/0/1584 MB | GPU: 0/0/4958 MB | Time 0:00:00.000 | (Consumed/Peaked/Used Total)


In [14]:
rnd1 = np.random.random()

CPU: 0/0/1584 MB | GPU: 0/0/4958 MB | Time 0:00:00.000 | (Consumed/Peaked/Used Total)


In [15]:
rnd2 = np.random.random()
assert rnd1 == rnd2, f"values should be the same rnd1={rnd1} rnd2={rnd2}"

CPU: 0/0/1584 MB | GPU: 0/0/4958 MB | Time 0:00:00.046 | (Consumed/Peaked/Used Total)


In [16]:
del exp2 # cleanup

CPU: 0/0/1584 MB | GPU: 0/0/4958 MB | Time 0:00:00.023 | (Consumed/Peaked/Used Total)


In [ ]:
%%javascript # prevent committing an unsaved notebook
_=IPython.notebook.save_notebook()